In [1]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

In [2]:
import zipfile

zip_file_uk = "/content/drive/MyDrive/data/cc.uk.300.vec.zip"
zip_file_ru = "/content/drive/MyDrive/data/cc.ru.300.vec.zip"

extract_dir_uk = "uk_embeddings"
extract_dir_ru = "ru_embeddings"

with zipfile.ZipFile(zip_file_uk, 'r') as zip_ref:
    zip_ref.extractall(extract_dir_uk)

with zipfile.ZipFile(zip_file_ru, 'r') as zip_ref:
    zip_ref.extractall(extract_dir_ru)


In [3]:
uk_emb = KeyedVectors.load_word2vec_format("uk_embeddings/cc.uk.300.vec")
ru_emb = KeyedVectors.load_word2vec_format("ru_embeddings/cc.ru.300.vec")

In [4]:
ru_emb.most_similar([ru_emb["август"]], topn=10)
uk_emb.most_similar([uk_emb["серпень"]])
ru_emb.most_similar([uk_emb["серпень"]])

[('Недопустимость', 0.24435284733772278),
 ('конструктивность', 0.23293082416057587),
 ('офор', 0.23256804049015045),
 ('deteydlya', 0.230317160487175),
 ('пресечении', 0.22632381319999695),
 ('одностороннего', 0.22608886659145355),
 ('подход', 0.2230587750673294),
 ('иболее', 0.22003726661205292),
 ('2015Александр', 0.21872766315937042),
 ('конструктивен', 0.21796567738056183)]

In [5]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [6]:
uk_ru_train, X_train, Y_train = load_word_pairs("/content/drive/MyDrive/ukr_rus.train.txt")

In [7]:
uk_ru_test, X_test, Y_test = load_word_pairs("/content/drive/MyDrive/ukr_rus.test.txt")

In [8]:
from sklearn.linear_model import LinearRegression
mapping = LinearRegression(fit_intercept=False)
mapping.fit(X_train, Y_train)


august = mapping.predict(uk_emb["серпень"].reshape(1, -1))


ru_emb.most_similar(august)


[('апрель', 0.8541285991668701),
 ('июнь', 0.8411202430725098),
 ('март', 0.839699387550354),
 ('сентябрь', 0.835986852645874),
 ('февраль', 0.8329297304153442),
 ('октябрь', 0.8311845660209656),
 ('ноябрь', 0.8278923630714417),
 ('июль', 0.8234528303146362),
 ('август', 0.812049925327301),
 ('декабрь', 0.803900420665741)]

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
    pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
    mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
    topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
    precision_val, float number, total number of words for those we can find the right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        similarities = cosine_similarity([mapped_vectors[i]], ru_emb.vectors)
        top_similar_indices = np.argsort(similarities, axis=1)[0][-topn:]
        top_similar_words = [ru_emb.index_to_key[idx] for idx in top_similar_indices]
        if ru in top_similar_words:
            num_matches += 1
    precision_val = num_matches / len(pairs)
    return precision_val

In [10]:
assert precision([("серпень", "август")], august, topn=10) == 1.0


In [11]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [12]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [13]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

assert precision_top1 >= 0.635
assert precision_top5 >= 0.813

In [14]:
def learn_transform(X_train, Y_train):
    """
    :param X_train: Source embeddings (uk_emb)
    :param Y_train: Target embeddings (ru_emb)
    :returns: Orthogonal transformation matrix W
    """
    covariance_matrix = np.dot(X_train.T, Y_train)
    U, _, Vt = np.linalg.svd(covariance_matrix)
    W = np.dot(U, Vt)

    return W


In [15]:
W = learn_transform(X_train, Y_train)

In [16]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8237906694412231),
 ('сентябрь', 0.8049713373184204),
 ('март', 0.8025653958320618),
 ('июнь', 0.8021842241287231),
 ('октябрь', 0.8001735806465149),
 ('ноябрь', 0.7934483289718628),
 ('февраль', 0.7914120554924011),
 ('июль', 0.7908109426498413),
 ('август', 0.7891016602516174),
 ('декабрь', 0.7686373591423035)]

In [17]:
assert precision(uk_ru_test, np.matmul(X_test, W)) >= 0.653
assert precision(uk_ru_test, np.matmul(X_test, W), 5) >= 0.824

In [18]:
with open("/content/drive/MyDrive/fairy_tale.txt", "r") as inpf:
    uk_sentences = [line.rstrip().lower() for line in inpf]

In [19]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)
    """
    words = sentence.split()

    translated_words = []
    for word in words:
        if word in uk_emb:
            uk_embedding = uk_emb[word]
            transformed_embedding = np.dot(uk_embedding, W)
            nearest_russian_word = ru_emb.similar_by_vector(transformed_embedding, topn=1)[0][0]
            translated_words.append(nearest_russian_word)
        else:
            translated_words.append(word)
    translation = ' '.join(translated_words)

    return translation

In [20]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

In [21]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

src: лисичка - сестричка і вовк - панібрат
dst: лисичка – сестричка и волк – панібрат

src: як була собі лисичка та зробила хатку, та й живе. а це приходять холоди. от лисичка замерзла та й побігла в село вогню добувать, щоб витопити. прибігає до одної баби та й каже:
dst: как была себе лисичка и сделала хатку, и и живе. а оно приходят холоди. из лисичка замерзла и и побежала во село огня добувать, чтобы витопити. прибегает к одной бабы и и каже:

src: — здорові були, бабусю! з неділею... позичте мені огню, я вам одслужу.
dst: — здоровые були, бабусю! со неділею... позичте мне огню, мной тебе одслужу.

src: — добре, — каже, — лисичко - сестричко. сідай погрійся трохи, поки я пиріжечки повибираю з печі!
dst: — добре, — каже, — лисичко – сестричко. садись погрійся трохи, пока мной пирожки повибираю со печі!

src: а баба макові пиріжки пекла. от баба вибирає пиріжки та на столі кладе, щоб прохололи; а лисичка підгляділа та за пиріг, та з хати... виїла мачок із середини, а туди напхала смі